In [1]:
%pylab inline
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


In [2]:
import tensorflow as tf
from tensorflow.models.rnn import rnn, rnn_cell
import numpy as np
import numpy.random as rng


In [3]:
import time

In [4]:
import pandas.io.data as web
from pykalman import KalmanFilter
import numpy as np
import pandas as pd

In [5]:
def get_prices(symbol):
    start, end = '2007-05-02', '2016-04-11'
    data = web.DataReader(symbol, 'yahoo', start, end)
    data=pd.DataFrame(data)
    prices=data['Adj Close']
    #prices=np.asarray(list(prices))
    prices=prices.astype(float)
    return prices

gs = get_prices('GS')

def get_returns(prices):
    return (prices-prices.shift(-1))/prices

rets = get_returns(gs)
rets = rets[:-1]
len(rets)

2251

In [6]:
def lstm_iterator(raw_data, batch_size, num_steps):
 
  raw_data = np.array(raw_data, dtype=np.float32)

  data_len = len(raw_data)
  batch_len = data_len // batch_size
  data = np.zeros([batch_size, batch_len], dtype=np.float32)
  for i in range(batch_size):
    data[i] = raw_data[batch_len * i:batch_len * (i + 1)]

  epoch_size = (batch_len - 1) // num_steps

  if epoch_size == 0:
    raise ValueError("epoch_size == 0, decrease batch_size or num_steps")

  for i in range(epoch_size):
    x = data[:, i*num_steps:(i+1)*num_steps]
    y = data[:, i*num_steps+1:(i+1)*num_steps+1]
    yield (x, y)

In [7]:
def run_epoch(session, m, data, eval_op, verbose=False):
  """Runs the model on the given data."""
  epoch_size = ((len(data) // m.batch_size) - 1) // m.num_steps
  start_time = time.time()
  costs = 0.0
  iters = 0
  state = m.initial_state.eval()
  for step, (x, y) in enumerate(lstm_iterator(data, m.batch_size,
                                                    m.num_steps)):
    cost, state, _ = session.run([m.cost, m.final_state, eval_op],
                                 {m.input_data: x,
                                  m.targets: y,
                                  m.initial_state: state})
    costs += cost
    iters += m.num_steps

  ''' if verbose and step % (epoch_size // 10) == 10:
      print("%.3f perplexity: %.3f speed: %.0f wps" %
            (step * 1.0 / epoch_size, np.exp(costs / iters),
             iters * m.batch_size / (time.time() - start_time)))
  '''
  return np.exp(costs / iters)



In [26]:
x = 'sdf'
reversed(x)

In [25]:
''.join(l)

'abcdef'

In [53]:
def addElement(element,list_val=list()): 
    list_val.append(element) 
    return list_val 
for i in range(10): 
    if i == 0: 
        list_val = addElement(i) 
    else: addElement(i)

In [54]:
list_val

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [45]:
x = [int(z) for z in list(x)]

In [ ]:
def foo(x,y):
    x = [int(z) for z in list(x)]
    return sum([x[i] == y[-(i+1)] for i in range(len(x))]) == len(x)

In [51]:
sum([x[i] == y[-(i+1)] for i in range(len(x))]) == len(x)

True

In [27]:
def foo(ls):
    ret  = []
    for string in ls:
        ret.append(string[::-1])
    return ''.join(ret)

In [28]:
foo(l)

'badcef'

In [8]:
class SmallConfig(object):
    """Small config."""
    init_scale = 0.1
    learning_rate = 1.0
    max_grad_norm = 5
    num_layers = 2
    num_steps = 25
    hidden_size = 200
    max_epoch = 4
    max_max_epoch = 13
    keep_prob = 1.0
    lr_decay = 0.5
    batch_size = 20
    #vocab_size = 10000

def get_config():
    return SmallConfig()

In [268]:
class LSTMModel(object):
  """The PTB model."""

  def __init__(self, is_training, config):
    self.batch_size = batch_size = config.batch_size
    self.num_steps = num_steps = config.num_steps
    size = config.hidden_size

    self._input_data = tf.placeholder(tf.float32, [batch_size, num_steps])
    self._targets = tf.placeholder(tf.float32, [batch_size, num_steps])
 

    z = tf.zeros([20])
    z = tf.reshape(z, [-1, 1])
    print(session.run(z).shape)
    
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(size, forget_bias=0.0)
    
    cell = tf.nn.rnn_cell.MultiRNNCell([lstm_cell] * config.num_layers)

    self._initial_state = cell.zero_state(batch_size, tf.float32)
    
    cell = tf.nn.rnn_cell.InputProjectionWrapper(cell,  size)
    
    #maybe:
    cell = tf.nn.rnn_cell.OutputProjectionWrapper(cell, 1)
    
    

    

    outputs = []
    state = self._initial_state
    #print(session.run(inputs))
    print(state)
    print(self._input_data)       #####
    with tf.variable_scope("RNN"):
      for time_step in range(num_steps):
        if time_step > 0: tf.get_variable_scope().reuse_variables()
        (cell_output, state) = cell(tf.reshape(self._input_data[:, time_step], [-1, 1]), state)    #
        outputs.append(cell_output)

    output = tf.reshape(tf.concat(1, outputs), [-1])
    loss = tf.nn.l2_loss(output - tf.reshape(self._targets, [-1]))
        
        
    '''output = tf.reshape(tf.concat(1, outputs), [-1, size])    #?????
    softmax_w = tf.get_variable("softmax_w", [size, 1])
    softmax_b = tf.get_variable("softmax_b", [1])
    logits = tf.matmul(output, softmax_w) + softmax_b

    loss = tf.nn.l2_loss(logits - tf.reshape(self._targets, [-1]))'''
    
    #loss = tf.nn.seq2seq.sequence_loss_by_example(
     #   [logits],
      #  [tf.reshape(self._targets, [-1])],
       # [tf.ones([batch_size * num_steps])],1)
    self._cost = cost = tf.reduce_sum(loss) / batch_size
    self._final_state = state

    if not is_training:
      return

    self._lr = tf.Variable(0.0, trainable=False)
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars),
                                      config.max_grad_norm)
    optimizer = tf.train.GradientDescentOptimizer(self.lr)
    self._train_op = optimizer.apply_gradients(zip(grads, tvars))

  def assign_lr(self, session, lr_value):
    session.run(tf.assign(self.lr, lr_value))

  @property
  def input_data(self):
    return self._input_data

  @property
  def targets(self):
    return self._targets

  @property
  def initial_state(self):
    return self._initial_state

  @property
  def cost(self):
    return self._cost

  @property
  def final_state(self):
    return self._final_state

  @property
  def lr(self):
    return self._lr

  @property
  def train_op(self):
    return self._train_op



In [273]:

raw_data = rets
#train_data, valid_data, test_data, _ = raw_data
train_data =valid_data = test_data =  raw_data

config = eval_config = get_config()
#eval_config = get_config()
#eval_config.batch_size = 1
#eval_config.num_steps = 1

with tf.Graph().as_default(), tf.Session() as session:
    initializer = tf.random_uniform_initializer(-config.init_scale,
                                            config.init_scale)
    with tf.variable_scope("model", reuse=None, initializer=initializer):
        m = LSTMModel(is_training=True, config=config)
    with tf.variable_scope("model", reuse=True, initializer=initializer):
        mvalid = LSTMModel(is_training=False, config=config)
        mtest = LSTMModel(is_training=False, config=eval_config)

    tf.initialize_all_variables().run()

    for i in range(config.max_max_epoch):
        lr_decay = config.lr_decay ** max(i - config.max_epoch, 0.0)
        m.assign_lr(session, config.learning_rate * lr_decay)

        print("Epoch: %d Learning rate: %.3f" % (i + 1, session.run(m.lr)))
        train_perplexity = run_epoch(session, m, train_data, m.train_op,
                                   verbose=True)

        print("Epoch: %d Train Perplexity: %.3f" % (i + 1, train_perplexity))
        valid_perplexity = run_epoch(session, mvalid, valid_data, tf.no_op())
        print("Epoch: %d Valid Perplexity: %.3f" % (i + 1, valid_perplexity))

    test_perplexity = run_epoch(session, mtest, test_data, tf.no_op())
    print("Test Perplexity: %.3f" % test_perplexity)


(20, 1)
Tensor("model/zeros_1:0", shape=TensorShape([Dimension(None), Dimension(800)]), dtype=float32)
Tensor("model/Placeholder:0", shape=TensorShape([Dimension(20), Dimension(25)]), dtype=float32)
(20, 1)
Tensor("model_1/zeros_1:0", shape=TensorShape([Dimension(None), Dimension(800)]), dtype=float32)
Tensor("model_1/Placeholder:0", shape=TensorShape([Dimension(20), Dimension(25)]), dtype=float32)
(20, 1)
Tensor("model_1/zeros_3:0", shape=TensorShape([Dimension(None), Dimension(800)]), dtype=float32)
Tensor("model_1/Placeholder_2:0", shape=TensorShape([Dimension(20), Dimension(25)]), dtype=float32)
Epoch: 1 Learning rate: 1.000
Epoch: 1 Train Perplexity: 2.812
Epoch: 1 Valid Perplexity: 1.974
Epoch: 2 Learning rate: 1.000
Epoch: 2 Train Perplexity: 362.734
Epoch: 2 Valid Perplexity: 9.814
Epoch: 3 Learning rate: 1.000
Epoch: 3 Train Perplexity: 40.849
Epoch: 3 Valid Perplexity: 1.843
Epoch: 4 Learning rate: 1.000
Epoch: 4 Train Perplexity: 671582.534
Epoch: 4 Valid Perplexity: 17.036


In [228]:
tf.nn.embedding_lookup?

In [226]:
'''class PTBModel(object):
  """The PTB model."""

  def __init__(self, is_training, config):
    self.batch_size = batch_size = config.batch_size
    self.num_steps = num_steps = config.num_steps
    size = config.hidden_size
    #vocab_size = config.vocab_size

    self._input_data = tf.placeholder(tf.int32, [batch_size, num_steps])
    self._targets = tf.placeholder(tf.int32, [batch_size, num_steps])

    # Slightly better results can be obtained with forget gate biases
    # initialized to 1 but the hyperparameters of the model would need to be
    # different than reported in the paper.
    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(size, forget_bias=0.0)
    #if is_training and config.keep_prob < 1:
     # lstm_cell = tf.nn.rnn_cell.DropoutWrapper(
      #    lstm_cell, output_keep_prob=config.keep_prob)
    cell = tf.nn.rnn_cell.MultiRNNCell([lstm_cell] * config.num_layers)

    self._initial_state = cell.zero_state(batch_size, tf.float32)

    #with tf.device("/cpu:0"):
     # embedding = tf.get_variable("embedding", [vocab_size, size])
      #inputs = tf.nn.embedding_lookup(embedding, self._input_data)
    

    #if is_training and config.keep_prob < 1:
     # inputs = tf.nn.dropout(inputs, config.keep_prob)

    # Simplified version of tensorflow.models.rnn.rnn.py's rnn().
    # This builds an unrolled LSTM for tutorial purposes only.
    # In general, use the rnn() or state_saving_rnn() from rnn.py.
    #
    # The alternative version of the code below is:
    #
    # from tensorflow.models.rnn import rnn
    # inputs = [tf.squeeze(input_, [1])
    #           for input_ in tf.split(1, num_steps, inputs)]
    # outputs, state = rnn.rnn(cell, inputs, initial_state=self._initial_state)
    outputs = []
    state = self._initial_state
    with tf.variable_scope("RNN"):
      for time_step in range(num_steps):
        if time_step > 0: tf.get_variable_scope().reuse_variables()
        (cell_output, state) = cell(inputs[:, time_step, :], state)
        outputs.append(cell_output)

    output = tf.reshape(tf.concat(1, outputs), [-1, size])
    softmax_w = tf.get_variable("softmax_w", [size, vocab_size])
    softmax_b = tf.get_variable("softmax_b", [vocab_size])
    logits = tf.matmul(output, softmax_w) + softmax_b
    loss = tf.nn.seq2seq.sequence_loss_by_example(
        [logits],
        [tf.reshape(self._targets, [-1])],
        [tf.ones([batch_size * num_steps])])
    self._cost = cost = tf.reduce_sum(loss) / batch_size
    self._final_state = state

    if not is_training:
      return

    self._lr = tf.Variable(0.0, trainable=False)
    tvars = tf.trainable_variables()
    grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars),
                                      config.max_grad_norm)
    optimizer = tf.train.GradientDescentOptimizer(self.lr)
    self._train_op = optimizer.apply_gradients(zip(grads, tvars))

  def assign_lr(self, session, lr_value):
    session.run(tf.assign(self.lr, lr_value))

  @property
  def input_data(self):
    return self._input_data

  @property
  def targets(self):
    return self._targets

  @property
  def initial_state(self):
    return self._initial_state

  @property
  def cost(self):
    return self._cost

  @property
  def final_state(self):
    return self._final_state

  @property
  def lr(self):
    return self._lr

  @property
  def train_op(self):
    return self._train_op

'''

'class PTBModel(object):\n  """The PTB model."""\n\n  def __init__(self, is_training, config):\n    self.batch_size = batch_size = config.batch_size\n    self.num_steps = num_steps = config.num_steps\n    size = config.hidden_size\n    #vocab_size = config.vocab_size\n\n    self._input_data = tf.placeholder(tf.int32, [batch_size, num_steps])\n    self._targets = tf.placeholder(tf.int32, [batch_size, num_steps])\n\n    # Slightly better results can be obtained with forget gate biases\n    # initialized to 1 but the hyperparameters of the model would need to be\n    # different than reported in the paper.\n    lstm_cell = tf.nn.rnn_cell.BasicLSTMCell(size, forget_bias=0.0)\n    #if is_training and config.keep_prob < 1:\n     # lstm_cell = tf.nn.rnn_cell.DropoutWrapper(\n      #    lstm_cell, output_keep_prob=config.keep_prob)\n    cell = tf.nn.rnn_cell.MultiRNNCell([lstm_cell] * config.num_layers)\n\n    self._initial_state = cell.zero_state(batch_size, tf.float32)\n\n    #with tf.device